In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats

In [2]:
train = pd.read_csv('dataset/train.csv', nrows=5000000)

In [3]:
train.head()

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,0,3,2,50,628,1
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,1,1,2,50,628,1
2,2014-08-11 08:24:33,2,3,66,348,48862,2234.2641,12,0,0,...,0,1,8250,1,0,1,2,50,628,1
3,2014-08-09 18:05:16,2,3,66,442,35390,913.1932,93,0,0,...,0,1,14984,1,0,1,2,50,1457,80
4,2014-08-09 18:08:18,2,3,66,442,35390,913.6259,93,0,0,...,0,1,14984,1,0,1,2,50,1457,21


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 24 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   date_time                  object 
 1   site_name                  int64  
 2   posa_continent             int64  
 3   user_location_country      int64  
 4   user_location_region       int64  
 5   user_location_city         int64  
 6   orig_destination_distance  float64
 7   user_id                    int64  
 8   is_mobile                  int64  
 9   is_package                 int64  
 10  channel                    int64  
 11  srch_ci                    object 
 12  srch_co                    object 
 13  srch_adults_cnt            int64  
 14  srch_children_cnt          int64  
 15  srch_rm_cnt                int64  
 16  srch_destination_id        int64  
 17  srch_destination_type_id   int64  
 18  is_booking                 int64  
 19  cnt                        int64  
 20  ho

Add time deltas of stay. Also add a month of check-in since some destinations are only interesting in specific months.

In [5]:
def dates_magic(df):
    df['srch_ci'] = pd.to_datetime(df['srch_ci'], errors='coerce')
    df['srch_co'] = pd.to_datetime(df['srch_co'], errors='coerce')
    df['date_time'] = pd.to_datetime(df['date_time'], errors='coerce')
    
    df['stay_duration'] = (df['srch_co'] - df['srch_ci']).apply(lambda x: x.days if not pd.isna(x) else x)
    df['days2ci'] = (df['srch_ci'] - df['date_time']).apply(lambda x: x.days if not pd.isna(x) else x)
    df['days2co'] = (df['srch_co'] - df['date_time']).apply(lambda x: x.days if not pd.isna(x) else x)
    df['ci_month'] = df['srch_ci'].apply(lambda x: x.month)

    # month_val = df['ci_month'].value_counts().idxmax()
    # dur_val = df['stay_duration'].value_counts().idxmax()
    # days2_val = df['days2cin'].value_counts().idxmax()
    df['stay_duration'] = df['stay_duration'].fillna(-1)
    df['days2ci'] = df['days2ci'].fillna(-1)
    df['days2co'] = df['days2co'].fillna(-1)
    df['ci_month'] = df['ci_month'].fillna(-1)

    df['srch_ci_timestamp'] = df['srch_ci'].apply(lambda x: x.timestamp() if not pd.isna(x) else x).astype(np.int64)
    df['srch_co_timestamp'] = df['srch_co'].apply(lambda x: x.timestamp() if not pd.isna(x) else x).astype(np.int64)
    df['date_time_timestamp'] = df['date_time'].apply(lambda x: x.timestamp() if not pd.isna(x) else x).astype(np.int64)
    
    df['srch_ci_timestamp'] = df['srch_ci_timestamp'].fillna(-1)
    df['srch_co_timestamp'] = df['srch_ci_timestamp'].fillna(-1)
    df['date_time_timestamp'] = df['srch_ci_timestamp'].fillna(-1)

    df['srch_ci'] = df['srch_ci'].astype(str)
    df['srch_co'] = df['srch_co'].astype(str)
    df['date_time'] = df['date_time'].astype(str)
    df['stay_duration'] = df['stay_duration'].astype(np.int)
    df['days2ci'] = df['days2ci'].astype(np.int)
    df['ci_month'] = df['ci_month'].astype(np.int)

In [6]:
dates_magic(train)

In [7]:
train.head(20)

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,hotel_country,hotel_market,hotel_cluster,stay_duration,days2ci,days2co,ci_month,srch_ci_timestamp,srch_co_timestamp,date_time_timestamp
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,...,50,628,1,4,15,19,8,1409097600,1409097600,1409097600
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,50,628,1,4,17,21,8,1409270400,1409270400,1409270400
2,2014-08-11 08:24:33,2,3,66,348,48862,2234.2641,12,0,0,...,50,628,1,4,17,21,8,1409270400,1409270400,1409270400
3,2014-08-09 18:05:16,2,3,66,442,35390,913.1932,93,0,0,...,50,1457,80,5,105,110,11,1416700800,1416700800,1416700800
4,2014-08-09 18:08:18,2,3,66,442,35390,913.6259,93,0,0,...,50,1457,21,5,105,110,11,1416700800,1416700800,1416700800
5,2014-08-09 18:13:12,2,3,66,442,35390,911.5142,93,0,0,...,50,1457,92,5,105,110,11,1416700800,1416700800,1416700800
6,2014-07-16 09:42:23,2,3,66,189,10067,NaN,501,0,0,...,50,675,41,1,15,16,8,1406851200,1406851200,1406851200
7,2014-07-16 09:45:48,2,3,66,189,10067,NaN,501,0,1,...,50,675,41,1,15,16,8,1406851200,1406851200,1406851200
8,2014-07-16 09:52:11,2,3,66,189,10067,NaN,501,0,0,...,50,675,69,1,15,16,8,1406851200,1406851200,1406851200
9,2014-07-16 09:55:24,2,3,66,189,10067,NaN,501,0,0,...,50,675,70,1,15,16,8,1406851200,1406851200,1406851200


Add distance features. 

In [8]:
def dist_magic(df): 
    # Fill nulls with average distance
    orig_dist_val = df['orig_destination_distance'].mean()
    df['orig_destination_distance'].fillna(orig_dist_val, inplace=True)


In [9]:
dist_magic(train)

Number of people, rooms, etc.

In [10]:
train['srch_adults_cnt'].value_counts()

2    3277554
1    1073142
3     275018
4     263937
6      45165
5      37041
8      10948
0       9709
7       5749
9       1737
Name: srch_adults_cnt, dtype: int64

In [11]:
train['srch_children_cnt'].value_counts()

0    3962543
1     551350
2     396717
3      65388
4      18727
5       2524
6       1834
7        439
8        344
9        134
Name: srch_children_cnt, dtype: int64

In [12]:
train['srch_rm_cnt'].value_counts()

1    4578461
2     336200
3      55009
4      14656
5       6185
8       4562
6       3200
7       1619
0        108
Name: srch_rm_cnt, dtype: int64

In [13]:
def cnt_magic(df): 
    df['srch_people_cnt'] = (df['srch_adults_cnt'] + df['srch_children_cnt'])


In [14]:
cnt_magic(train)

Remove different columns. Dates, distances, user ids are useless. Some other features aren't really informative too. So maybe remove those too.

In [15]:
columns = ['srch_ci', 'srch_co', 'date_time']
train.drop(columns, axis=1, inplace=True)

In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 29 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   site_name                  int64  
 1   posa_continent             int64  
 2   user_location_country      int64  
 3   user_location_region       int64  
 4   user_location_city         int64  
 5   orig_destination_distance  float64
 6   user_id                    int64  
 7   is_mobile                  int64  
 8   is_package                 int64  
 9   channel                    int64  
 10  srch_adults_cnt            int64  
 11  srch_children_cnt          int64  
 12  srch_rm_cnt                int64  
 13  srch_destination_id        int64  
 14  srch_destination_type_id   int64  
 15  is_booking                 int64  
 16  cnt                        int64  
 17  hotel_continent            int64  
 18  hotel_country              int64  
 19  hotel_market               int64  
 20  ho

In [17]:
def aggregate_magic(df):
    df['previous_queries_count'] = df \
        .sort_values(by='date_time_timestamp', kind='mergesort') \
        .groupby(['user_id']) \
        .cumcount() + 0.001

    for agg_value in ['continent', 'country', 'market']:
        df[f'same_{agg_value}_counts'] = df \
            .sort_values(by='date_time_timestamp', kind='mergesort') \
            .groupby(['user_id', f'hotel_{agg_value}']) \
            .cumcount()

        df[f'same_{agg_value}_ratio'] = df[f'same_{agg_value}_counts'] / df['previous_queries_count']

In [18]:
aggregate_magic(train)

In [19]:
train.head(20)

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,...,srch_co_timestamp,date_time_timestamp,srch_people_cnt,previous_queries_count,same_continent_counts,same_continent_ratio,same_country_counts,same_country_ratio,same_market_counts,same_market_ratio
0,2,3,66,348,48862,2234.264100,12,0,1,9,...,1409097600,1409097600,2,0.001,0,0.000000,0,0.000000,0,0.000000
1,2,3,66,348,48862,2234.264100,12,0,1,9,...,1409270400,1409270400,2,1.001,1,0.999001,1,0.999001,1,0.999001
2,2,3,66,348,48862,2234.264100,12,0,0,9,...,1409270400,1409270400,2,2.001,2,0.999500,2,0.999500,2,0.999500
3,2,3,66,442,35390,913.193200,93,0,0,3,...,1416700800,1416700800,2,0.001,0,0.000000,0,0.000000,0,0.000000
4,2,3,66,442,35390,913.625900,93,0,0,3,...,1416700800,1416700800,2,1.001,1,0.999001,1,0.999001,1,0.999001
5,2,3,66,442,35390,911.514200,93,0,0,3,...,1416700800,1416700800,2,2.001,2,0.999500,2,0.999500,2,0.999500
6,2,3,66,189,10067,1972.534634,501,0,0,2,...,1406851200,1406851200,2,0.001,0,0.000000,0,0.000000,0,0.000000
7,2,3,66,189,10067,1972.534634,501,0,1,2,...,1406851200,1406851200,2,1.001,1,0.999001,1,0.999001,1,0.999001
8,2,3,66,189,10067,1972.534634,501,0,0,2,...,1406851200,1406851200,2,2.001,2,0.999500,2,0.999500,2,0.999500
9,2,3,66,189,10067,1972.534634,501,0,0,2,...,1406851200,1406851200,2,3.001,3,0.999667,3,0.999667,3,0.999667


In [20]:
destination_info = pd.read_csv('dataset/destinations.csv')

In [21]:
destination_info

,srch_destination_id,d1,d2,d3,d4,d5,d6,d7,d8,d9,...,d140,d141,d142,d143,d144,d145,d146,d147,d148,d149
0,0,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-1.897627,-2.198657,-2.198657,-1.897627,...,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657
1,1,-2.181690,-2.181690,-2.181690,-2.082564,-2.181690,-2.165028,-2.181690,-2.181690,-2.031597,...,-2.165028,-2.181690,-2.165028,-2.181690,-2.181690,-2.165028,-2.181690,-2.181690,-2.181690,-2.181690
2,2,-2.183490,-2.224164,-2.224164,-2.189562,-2.105819,-2.075407,-2.224164,-2.118483,-2.140393,...,-2.224164,-2.224164,-2.196379,-2.224164,-2.192009,-2.224164,-2.224164,-2.224164,-2.224164,-2.057548
3,3,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.115485,-2.177409,-2.177409,-2.177409,...,-2.161081,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409
4,4,-2.189562,-2.187783,-2.194008,-2.171153,-2.152303,-2.056618,-2.194008,-2.194008,-2.145911,...,-2.187356,-2.194008,-2.191779,-2.194008,-2.194008,-2.185161,-2.194008,-2.194008,-2.194008,-2.188037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62101,64989,-2.145876,-2.158016,-2.216621,-2.158016,-2.006209,-2.102505,-2.216621,-2.157203,-2.216621,...,-2.216621,-2.158016,-2.158016,-2.216621,-2.216621,-2.216621,-2.216621,-2.216621,-2.216621,-2.152024
62102,64990,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.176802,-2.203479,-2.203479,-2.203479,...,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.008885
62103,64991,-2.190567,-2.208308,-2.208308,-2.208308,-2.208308,-2.135493,-2.208308,-2.208308,-1.909808,...,-2.188935,-2.208308,-2.175594,-2.208308,-2.208308,-2.199347,-2.208308,-2.208308,-2.208308,-2.208308
62104,64992,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,...,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686


In [22]:
train = train.join(destination_info, on='srch_destination_id', how='left', rsuffix='_destination')

In [23]:
train.head(20)

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,...,d140,d141,d142,d143,d144,d145,d146,d147,d148,d149
0,2,3,66,348,48862,2234.264100,12,0,1,9,...,-2.221012,-2.221012,-2.221012,-2.221012,-2.221012,-2.221012,-2.221012,-2.221012,-2.221012,-2.221012
1,2,3,66,348,48862,2234.264100,12,0,1,9,...,-2.221012,-2.221012,-2.221012,-2.221012,-2.221012,-2.221012,-2.221012,-2.221012,-2.221012,-2.221012
2,2,3,66,348,48862,2234.264100,12,0,0,9,...,-2.221012,-2.221012,-2.221012,-2.221012,-2.221012,-2.221012,-2.221012,-2.221012,-2.221012,-2.221012
3,2,3,66,442,35390,913.193200,93,0,0,3,...,-2.188108,-2.188108,-2.188108,-2.188108,-2.173472,-2.188108,-2.188108,-2.188108,-2.188108,-2.188108
4,2,3,66,442,35390,913.625900,93,0,0,3,...,-2.188108,-2.188108,-2.188108,-2.188108,-2.173472,-2.188108,-2.188108,-2.188108,-2.188108,-2.188108
5,2,3,66,442,35390,911.514200,93,0,0,3,...,-2.188108,-2.188108,-2.188108,-2.188108,-2.173472,-2.188108,-2.188108,-2.188108,-2.188108,-2.188108
6,2,3,66,189,10067,1972.534634,501,0,0,2,...,-2.175588,-2.175588,-2.175588,-2.175588,-2.175588,-2.175588,-2.175588,-2.175588,-2.175588,-2.175588
7,2,3,66,189,10067,1972.534634,501,0,1,2,...,-2.175588,-2.175588,-2.175588,-2.175588,-2.175588,-2.175588,-2.175588,-2.175588,-2.175588,-2.175588
8,2,3,66,189,10067,1972.534634,501,0,0,2,...,-2.175588,-2.175588,-2.175588,-2.175588,-2.175588,-2.175588,-2.175588,-2.175588,-2.175588,-2.175588
9,2,3,66,189,10067,1972.534634,501,0,0,2,...,-2.175588,-2.175588,-2.175588,-2.175588,-2.175588,-2.175588,-2.175588,-2.175588,-2.175588,-2.175588


In [24]:
train.sort_values(by='date_time_timestamp', ignore_index=True, inplace=True)

In [25]:
col_to_drop = [
    'srch_destination_id_destination', 'srch_destination_id', 
    'srch_ci_timestamp', 'srch_co_timestamp', 'date_time_timestamp',
    'previous_queries_count'
]

In [26]:
train.drop(col_to_drop, axis=1, inplace=True)

In [27]:
train.to_csv('../data/expedia/a_all.csv', index=False)